In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from splinter import Browser
import re

In [2]:
#URL to be scraped
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

In [3]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- Responsiveness -->
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <!-- Favicon -->
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/manifest.json" rel="manifest"/>
  <link color="#e48b55" href="/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="authenticity_token" name="csrf-param">
   <meta content="rD/oQJr6r7CDKTsAi09/7VirW/+moNjTRHepDzCAjRGhxg8lwrx+d7CKFDcv4DdtB2N4R+MRWBbqOmc0kJTjQQ==" name="csrf-to

In [5]:
#Narrow the div and class to see how best to loop through
results = soup.find_all('div', class_="content_title")
results

[<div class="content_title">
 <a href="/news/8749/nasa-readies-perseverance-mars-rovers-earthly-twin/">
 NASA Readies Perseverance Mars Rover's Earthly Twin 
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8716/nasa-to-broadcast-mars-2020-perseverance-launch-prelaunch-activities/">
 NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/">
 The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8692/nasa-to-hold-mars-2020-perseverance-rover-launch-briefing/">
 NASA to Hold Mars 2020 Perseverance Rover Launch Briefing
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/">
 Alabama High School Student Names NASA's Mars Helicopter
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8645/m

In [6]:
# A blank list to hold the headlines
news_titles = []
# Loop over div elements
for result in results:
    # Identify the anchor...
    if (result.a):
        # And the anchor has non-blank text...
        if (result.a.text):
            # Append thext to the list
            news_titles.append(result)
news_titles

[<div class="content_title">
 <a href="/news/8749/nasa-readies-perseverance-mars-rovers-earthly-twin/">
 NASA Readies Perseverance Mars Rover's Earthly Twin 
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8716/nasa-to-broadcast-mars-2020-perseverance-launch-prelaunch-activities/">
 NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/">
 The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8692/nasa-to-hold-mars-2020-perseverance-rover-launch-briefing/">
 NASA to Hold Mars 2020 Perseverance Rover Launch Briefing
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/">
 Alabama High School Student Names NASA's Mars Helicopter
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8645/m

In [7]:
finalnewstitles = []
# Print only the headlines
for x in range(6):
    var=news_titles[x].text
    newvar = var.strip('\n\n')
    finalnewstitles.append(newvar)
finalnewstitles

["NASA Readies Perseverance Mars Rover's Earthly Twin ",
 'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities',
 "The Launch Is Approaching for NASA's Next Mars Rover, Perseverance",
 'NASA to Hold Mars 2020 Perseverance Rover Launch Briefing',
 "Alabama High School Student Names NASA's Mars Helicopter",
 "Mars Helicopter Attached to NASA's Perseverance Rover"]

In [8]:
#Find classification for description paragraph below title
presults = soup.find_all('div', class_="rollover_description_inner")
presults

[<div class="rollover_description_inner">
 Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.
 </div>,
 <div class="rollover_description_inner">
 Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.
 </div>,
 <div class="rollover_description_inner">
 The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. 
 </div>,
 <div class="rollover_description_inner">
 Learn more about the agency's next Red Planet mission during a live event on June 17.
 </div>,
 <div class="rollover_description_inner">
 Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.
 </div>,
 <div class="rollover_d

In [9]:
news_p = []
# Loop through the div results to pull out just the text
for x in range(6):
    var=presults[x].text
    newvar = var.strip('\n\n')
    news_p.append(newvar)
news_p

["Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.",
 'Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.',
 "The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. ",
 "Learn more about the agency's next Red Planet mission during a live event on June 17.",
 "Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.",
 "The team also fueled the rover's sky crane to get ready for this summer's history-making launch."]

In [10]:

### JPL Mars Space Images - Featured Image

In [11]:

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
#executable_path = {'executable_path': 'chromedriver.exe'}
#browser = Browser('chrome', **executable_path, headless=False)
#browser.visit(url)
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [12]:
#Review html format
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 9]> <html class="no-js ie ie9" lang="en"> <![endif]-->
<!--[if IE 8]> <html class="no-js ie ie8" lang="en"> <![endif]-->
<html>
 <!-- START HEADER: "DEFAULT" -->
 <!-- Google Tag Manager -->
 <script>
  (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push(
{'gtm.start': new Date().getTime(),event:'gtm.js'}
);var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-NLDQZ25');
 </script>
 <!-- End Google Tag Manager -->
 <head>
  <meta charset="utf-8"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <title>
   Space Images
  </title>
  <link href="/assets/stylesheets/manifest.css" media="all"

In [13]:
imgs = soup.find_all('a', class_="fancybox")
imgs

[<a class="button fancybox" data-description="This view of the 'belly' and part of the 'head' of comet 67P/Churyumov-Gerasimenko indicates several morphologically different regions as seen by ESA's Rosetta spacecraft." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA18429_ip.jpg" data-link="/spaceimages/details.php?id=PIA18429" data-title="Distinct Terrains on Rosetta's Comet" id="full_image">
 					FULL IMAGE
 				  </a>,
 <a class="fancybox" data-description="This image from NASAs Mars Odyssey shows a section of Her Desher Vallis. This channel is located in Noachis Terra. Her Desher is the Egyptian word for Mars." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/largesize/PIA24088_hires.jpg" data-link="/spaceimages/details.php?id=PIA24088" data-thumbnail="/spaceimages/images/wallpaper/PIA24088-640x350.jpg" data-title="Her Desher Vallis">
 <div class="image_and_description_container">
 <div class="rollover_description">
 <h3 class

In [14]:

# pull image link
feat_img = []
for img in imgs:
    pic = img['data-fancybox-href']
    feat_img.append(pic)

featured_image_url = 'https://www.jpl.nasa.gov' + pic
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24014_hires.jpg'

In [15]:
!which chromedriver

which: no chromedriver in (/c/Users/seidi/.conda/envs/PythonData:/c/Users/seidi/.conda/envs/PythonData/Library/mingw-w64/bin:/c/Users/seidi/.conda/envs/PythonData/Library/usr/bin:/c/Users/seidi/.conda/envs/PythonData/Library/bin:/c/Users/seidi/.conda/envs/PythonData/Scripts:/c/Users/seidi/.conda/envs/PythonData/bin:/c/ProgramData/Anaconda2/Scripts/condabin:/c/Users/seidi/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/seidi/bin:/c/Program Files (x86)/Common Files/Oracle/Java/javapath:/c/ProgramData/Anaconda2:/c/ProgramData/Anaconda2/Library/mingw-w64/bin:/c/ProgramData/Anaconda2/Library/usr/bin:/c/ProgramData/Anaconda2/Library/bin:/c/ProgramData/Anaconda2/Scripts:/c/windows/system32:/c/windows:/c/windows/System32/Wbem:/c/windows/System32/WindowsPowerShell/v1.0:/c/windows/System32/OpenSSH:/c/Program Files/PuTTY:/c/Program Files/nodejs:/c/ProgramData/chocolatey/bin:/c/Program Files/MongoDB/Server/4.4/bin:/cmd:/c/Program Files/MySQL/MySQL Shell 8.0/bin:/c/

In [16]:

### Mars Facts
# Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing 
# facts about the planet including Diameter, Mass, etc.

#Use Pandas to convert the data to a HTML table string.

In [17]:
url = 'https://space-facts.com/mars/'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [18]:
#Review html format
#print(soup.prettify())

In [19]:
pip install html5lib

In [20]:
tables = pd.read_html(url)
tables[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [21]:
marsdf = tables[0]
marsdf

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [22]:
marsdf.columns = ['Stat', 'Measurement']
marsdf.head()

,Stat,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [23]:
s = pd.Series(marsdf['Stat'])
marsdf['Stat'] = s.str.strip(':')
marsdf

,Stat,Measurement
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.39 × 10^23 kg (0.11 Earths)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.38 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-87 to -5 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [24]:
marsdf = marsdf.set_index('Stat')
marsdf

,Measurement
Stat,
Equatorial Diameter,"6,792 km"
Polar Diameter,"6,752 km"
Mass,6.39 × 10^23 kg (0.11 Earths)
Moons,2 (Phobos & Deimos)
Orbit Distance,"227,943,824 km (1.38 AU)"
Orbit Period,687 days (1.9 years)
Surface Temperature,-87 to -5 °C
First Record,2nd millennium BC
Recorded By,Egyptian astronomers


In [25]:
#Use to_html method to generate HTML tables from df
html_table = marsdf.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Measurement</th>\n    </tr>\n    <tr>\n      <th>Stat</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody

In [26]:
#Save as html file
marsdf.to_html('mars_table.html')

In [27]:
### Mars Hemispheres
# Visit the USGS Astrogeology site [here]
# (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution 
# images for each of Mar's hemispheres.

# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the 
# hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one 
# dictionary for each hemisphere.

# Example:
#hemisphere_image_urls = [
#    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#    {"title": "Cerberus Hemisphere", "img_url": "..."},
#    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
#]

In [28]:
# Setting up windows browser with chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [29]:
# Setting url for alternate browser
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [30]:
nextpage_urls = []
imgtitles = []
base_url = 'https://astrogeology.usgs.gov'

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain hemisphere photo info
divs = soup.find_all('div', class_='description')

# Iterate through each div to pull titles and make list of hrefs to iterate through
counter = 0
for div in divs:
        # Use Beautiful Soup's find() method to navigate and retrieve attributes
    link = div.find('a')
    href=link['href']
    img_title = div.a.find('h3')
    img_title = img_title.text
    imgtitles.append(img_title)
    next_page = base_url + href
    nextpage_urls.append(next_page)
    counter = counter+1
    if (counter == 4):
        break
print(nextpage_urls)
print(imgtitles)

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']
['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']


In [31]:
# Creating loop for high resolution photo on next page

my_images=[]
for nextpage_url in nextpage_urls:
    url = nextpage_url
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    link2 = soup.find('img', class_="wide-image")
    forfinal = link2['src']
    full_img = base_url + forfinal
    my_images.append(full_img)
    nextpage_urls = []
my_images

['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [32]:

# Creating final list of dictionaries
# values - imgtitles and my_images
#ckeys- img_url and title
hemisphere_image_urls = []

cerberus = {'title':imgtitles[0], 'img_url': my_images[0]}
schiaparelli = {'title':imgtitles[1], 'img_url': my_images[1]}
syrtis = {'title':imgtitles[2], 'img_url': my_images[2]}
valles = {'title':imgtitles[3], 'img_url': my_images[3]}

hemisphere_image_urls = [cerberus, schiaparelli, syrtis, valles]
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
